In [ ]:
#default_exp s3_scrape

In [ ]:
#export

import boto3
import pandas as pd
import os
from PIL import Image

In [ ]:
#export

def scrape_bucket(bucket_name):
    s3_resource = boto3.resource('s3')
    s3_client = boto3.client('s3')
    bucket = s3_resource.Bucket(bucket_name)
    
    volume_ids = []
    titles = []
    volume_roots = []
    image_counts = []#
    has_jpg = []
    has_tif = []
    has_other = []
    other = []
    has_pdf = []
    has_metadata = []
    volume_metadata = []
    
    folders = ["jpg", "tif", "metadata"]    
    
    for obj in bucket.objects.all():            
        if (len(obj.key.split('/')) >= 5) and (obj.key.split('/')[4] not in volume_ids):            
            volume_ids.append(obj.key.split('/')[4])
            volume_root = obj.key[:obj.key.rfind('/')]
            volume_roots.append(volume_root)
            has_metadata.append(False)
            has_pdf.append(False)
            has_jpg.append(False)
            has_tif.append(False)
            has_other.append(False)
            other.append([])
            for volume_obj in bucket.objects.filter(Prefix = volume_root):
                if "DC.xml" in volume_obj.key:
                    has_metadata[-1] = True
                    s3_client.download_file(bucket.name, volume_obj.key, "temp.xml")
                    vol_dict = ssda_volume_xml_to_dict("temp.xml", volume_root)
                    volume_metadata.append(vol_dict)
                    titles.append(vol_dict["title"])
                    os.remove("temp.xml")
                elif "pdf" in volume_obj.key.lower():
                    has_pdf[-1] = True
                elif (has_jpg[-1] == False) and (volume_obj.key.lower().split('/')[5] == "jpg"):
                    has_jpg[-1] = True
                elif (has_tif[-1] == False) and (volume_obj.key.lower().split('/')[5] == "tif"):
                    has_tif[-1] = True
                elif (len(volume_obj.key.split('/')) > 6) and (volume_obj.key.lower().split('/')[5] not in folders):
                    has_other[-1] = True
                    other[-1].append(volume_obj.key.lower().split('/')[5])
                
            image_metadata = []
            prod_imgs = None
            if has_jpg[-1]:
                prod_imgs = "jpg"
            elif has_tif[-1]:
                prod_imgs = "tif"
                
            if prod_imgs == None:
                volume_metadata[-1]["images"] = []
                image_counts.append(0)
            else:                
                for image_obj in bucket.objects.filter(Prefix = volume_root + '/' + prod_imgs.upper()):
                    if ('.' + prod_imgs) in image_obj.key.lower():
                        file_name = image_obj.key[image_obj.key.rfind('/') + 1:image_obj.key.rfind('.')]
                        extension = image_obj.key[image_obj.key.rfind('.') + 1:]
                        temp_path = file_name + '.' + extension
                        s3_client.download_file(bucket.name, image_obj.key, temp_path)
                        im = Image.open(temp_path)
                        width, height = im.size
                        im.close()
                        os.remove(temp_path)
                        image = {"file_name": int(file_name), "extension": extension, "height": height, "width": width}
                        image_metadata.append(image)
                volume_metadata[-1]["images"] = image_metadata
                image_counts.append(len(image_metadata))            
            
            if has_metadata[-1] == False:
                titles.append("no title")           
                
    volumes_dict = {"id": volume_ids, "title": titles, "images": image_counts, "s3 root": volume_roots, "metadata": has_metadata, "has pdf": has_pdf, "has jpg": has_jpg, "has tif": has_tif, "has other": has_other, "other": other}
    volumes_df = pd.DataFrame.from_dict(volumes_dict)
    
    return volumes_df, volume_metadata

In [ ]:
#export

def ssda_volume_xml_to_dict(volume_xml, s3_path):
    import xml.etree.ElementTree as ET
    tree = ET.parse(volume_xml)
    root = tree.getroot()
    volume_dict = {}
    volume_dict["s3_path"] = s3_path
    for item in root:
        if item.text == None:
            volume_dict[item.tag] = None
        if item.text[0] == ' ':
            item.text = item.text[1:]        
        if item.tag == "subject":
            if "subject" in volume_dict:
                volume_dict["subject"].append(item.text.split("--"))
            else:
                volume_dict["subject"] = item.text.split("--")
        elif item.tag == "title":
            volume_dict["title"] = item.text
        elif item.tag == "contributor":
            if (item.text.find('(') != -1) and (item.text.find(')') != -1):
                name = item.text[:item.text.find('(')]
                role = item.text[item.text.find('(') + 1:item.text.find(')')]
            else:
                continue
            if "contributor" in volume_dict:
                volume_dict["contributor"].append({"name": name, "role": role})
            else:
                volume_dict["contributor"] = [{"name": name, "role": role}]
        elif item.tag == "identifier":
            volume_dict["identifier"] = item.text[item.text.find(':') + 1:]
        elif item.tag == "coverage":
            if ('.' in item.text) and (',' in item.text):
                if "coverage" in volume_dict:
                    volume_dict["coverage"]["coords"] = item.text
                else:
                    volume_dict["coverage"] = {"coords": item.text}
            elif "--" in item.text:
                places = item.text.split("--")
                if len(places) == 4:
                    if "coverage" in volume_dict:
                        volume_dict["coverage"]["country"] = places[1]
                        volume_dict["coverage"]["state"] = places[2]
                        volume_dict["coverage"]["city"] = places[3]
                    else:
                        volume_dict["coverage"] = {"country": places[1], "state": places[2], "city": places[3]}                
        elif item.tag == "source":
            if "coverage" in volume_dict:
                volume_dict["coverage"]["institution"] = item.text
            else:
                volume_dict["coverage"] = {"institution": item.text}
        elif ((item.tag == "type") and (item.text == "Text")) or (item.tag == "rights"):
            continue
        else:
            if item.tag in volume_dict:                
                volume_dict[item.tag].append(item.text)                
            else:
                volume_dict[item.tag] = [item.text]       
        
        if "coverage" not in volume_dict:
            volume_dict["coverage"] = {}
            if "country" not in volume_dict["coverage"]:            
                volume_dict["coverage"]["country"] = volume_dict["s3_path"].split('/')[0].replace('_', ' ')
                volume_dict["coverage"]["state"] = volume_dict["s3_path"].split('/')[1].replace('_', ' ')
                volume_dict["coverage"]["city"] = volume_dict["s3_path"].split('/')[2].replace('_', ' ')
        
    return volume_dict

In [ ]:
#no_test

volume_df, volume_metadata = scrape_bucket("ssda-assets")

In [ ]:
#no_test

volume_df.head()

,id,title,images,s3 root,metadata,has pdf,has jpg,has tif,has other,other
0,2223,Supplemento do Livro 8 para os assentos de óbi...,69,Brazil/Minas_Gerais/Nova_Lima/Nossa_Senhora_do...,True,True,True,False,False,[]


In [ ]:
#no_test

volume_metadata

[{'s3_path': 'Brazil/Minas_Gerais/Nova_Lima/Nossa_Senhora_do_Pilar/2223',
  'title': 'Supplemento do Livro 8 para os assentos de óbitos dos escravos da Companhia do Morro Velho',
  'coverage': {'country': 'Brazil',
   'state': 'Minas Gerais',
   'city': 'Nova Lima',
   'institution': 'Nossa Senhora do Pilar de Nova Lima',
   'coords': '-19.9854, -43.84711'},
  'creator': ['National Endowment for the Humanities Collaborative Research Grant'],
  'subject': ['Burials', 'Brazil', 'Modern'],
  'description': ['Full title: "Supplemento do Livro 8 desta Freguesia de Congonhas de Sabará para os assentos de óbitos dos escravos da Companhia do Morro Velho desde o primeiro de Abril do corrente ano de 1871." The volume contains burial records of enslaved individuals owned by the St. John d\'El Rey Mining Company (also known as the Companhia do Morro Velho). Good condition'],
  'publisher': ['Slave Societies Digital Archive'],
  'contributor': [{'name': 'Jane Landers', 'role': 'project director'},


In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()

Converted manifest_generation.ipynb.
Converted s3_scrape.ipynb.
